In [ ]:

import numpy as np
import tensorflow as tf

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform

import pydot
from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

import resnets_utils 


In [ ]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = "conv" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    F1, F2, F3 = filters
    X_shortcut = X
    # 卷积1
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1, 1), padding = "valid", 
              name = conv_name_base + "2a", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    # 卷积2
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same",
              name = conv_name_base + "2b", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    # 卷积3
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = "valid", 
              name = conv_name_base + "2c", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    
    # 相加
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X    

In [ ]:
tf.reset_default_graph()
with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float",[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    A = identity_block(A_prev,f=2,filters=[2,4,6],stage=1,block="a")
    
    test.run(tf.global_variables_initializer())
    out = test.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print("out = " + str(out[0][1][1][0]))
    
    test.close()


In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    conv_name_base = "conv" + str(stage) + block + "_brach"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    F1, F2, F3 = filters
    X_shortcut = X
    """
    strides = 1和valid，卷积前后尺寸不变
    """
    
    # 卷积1
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (s, s), padding = "valid",
              name = conv_name_base + "2a", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    # 卷积2
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = "same", 
              name = conv_name_base + "2b", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = Activation("relu")(X)
    # 卷积3
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = "valid", 
              name = conv_name_base + "2c", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + "2c")(X)
    # shortcut
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s, s), padding = "valid",
                       name = conv_name_base + "1", kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + "1")(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float",[3,4,4,6])
    X = np.random.randn(3,4,4,6)
    
    A = convolutional_block(A_prev,f=2,filters=[2,4,6],stage=1,block="a")
    test.run(tf.global_variables_initializer())
    
    out = test.run([A],feed_dict={A_prev:X,K.learning_phase():0})
    print("out = " + str(out[0][1][1][0]))
    
    test.close()


In [ ]:
def Resnet50(input_shape = (64, 64, 3), classes = 6):
    # 输入
    X_input = Input(input_shape)
    # ZERO PADDING
    X = ZeroPadding2D((3, 3))(X_input)
    # stage1
    X = Conv2D(filters = 64, kernel_size = (7, 7), strides = (2, 2),
              name = "conv1", kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name = "bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)
    # stage2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block = "a", s = 1)
    X = identity_block(X, f = 3, filters = [64, 64, 256], stage = 2, block = "b")
    X = identity_block(X, f = 3, filters = [64, 64, 256], stage = 2, block = "c")
    # stage3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = "a", s = 2)
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = "b")
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = "c")
    X = identity_block(X, f = 3, filters = [128, 128, 512], stage = 3, block = "d")
    # stage4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "a", s = 2)
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "b")
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "c")
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "d")
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "e")
    X = identity_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block = "f")
    # stage5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = "a", s = 2)
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = "b")
    X = identity_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block = "c")
    # stage6
    X = AveragePooling2D(pool_size = (2, 2), padding = "same")(X)
    # 输出
    X = Flatten()(X)
    X = Dense(classes, activation = "softmax", name = "fc" + str(classes),
             kernel_initializer = glorot_uniform(seed = 0))(X)
    # 创建模型
    model = Model(inputs = X_input, outputs = X, name = "ResNet50")
    
    return model

In [ ]:
model = Resnet50(input_shape = (64, 64, 3), classes = 6)
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = resnets_utils.load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = resnets_utils.convert_to_one_hot(Y_train_orig, 6).T
Y_test = resnets_utils.convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))


In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

In [ ]:
preds = model.evaluate(X_test,Y_test)

print("误差值 = " + str(preds[0]))
print("准确率 = " + str(preds[1]))



In [ ]:
#加载模型
model = load_model("ResNet50.h5") 
preds = model.evaluate(X_test,Y_test)
print("误差值 = " + str(preds[0]))
print("准确率 = " + str(preds[1]))


In [ ]:
# 测试自己的图片
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
img_path = "image/two.jpg"


img = image.load_img(img_path, target_size = (64, 64))  # 读出来是PIL类型
imshow(img)
print("(image.load_img),img.shape = {} and img.type = {}".format(None, type(img)))
img = image.img_to_array(img)
print("(image.img_to_array),img.shape = {} and img.type = {}".format(img.shape, type(img)))
img = np.expand_dims(img, axis = 0)
print("(np.expand_dims),img.shape = {} and img.type = {}".format(img.shape, type(img)))
img = preprocess_input(img)
print("(preprocess_input),img.shape = {} and img.type = {}".format(img.shape, type(img)))
print("{}".format('-'*10))
print(model.predict(img))
#img = scipy.misc.imread(img_path)


In [ ]:
model.summary()

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))